## Temporal Instanton Problem Instance

Let's make up some simple numbers and make an instance of the temporal instanton problem.

In [1]:
include("../src/TemporalInstanton.jl")
include("../src/TrustRegionSubproblem.jl")

using TemporalInstanton, TrustRegionSubproblem

In [2]:
Ridx = [1,2]
Y = [1 -2 -3;
    -2 2 -4;
    -3 -4 3]
slack = 3
k = [0.25,0.25,0.50]
tau = 2
line = (1,2)
G0 = [0.8,0.9,0.8, 0,0.5,0.5, 0.2,0.3,0.4]
D = flipud(G0)

kQobj = 0
kQtheta = 10

# Create instance of instanton problem:
Qobj,A,B,Qtheta = temporalInstanton(Ridx,Y,slack,k,tau,line,G0,D)

(
18x18 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

## Solving the trust region subproblem

In [6]:
# Step 1: find point in set Ax = b closest to origin.
x_star = min_norm(A,b)

18-element Array{FloatingPoint,1}:
 -0.161538   
 -0.238462   
  7.31897e-17
  0.0307692  
 -1.2        
  0.568182   
  0.431818   
  1.34407e-16
  0.0909091  
 -1.0        
  0.4        
  0.4        
  2.47228e-17
  1.07369e-16
  0.8        
  0.0615385  
  0.128565   
  3.33067e-16

In [7]:
# Step 2: Translate quadratics by x_star found in Step 1.
Qobj_trans,Qobj_q_trans,kQobj_trans = translate_quadratic(Qobj,0,kQobj,x_star)
Qtheta_trans,Qtheta_q_trans,kQtheta_trans = translate_quadratic(Qtheta,0,kQtheta,x_star)

(
18x18 sparse matrix with 3 Float64 entries:
	[16, 16]  =  1.0
	[17, 17]  =  1.0
	[18, 18]  =  1.0,

18x1 Array{FloatingPoint,2}:
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
 -0.123077   
 -0.25713    
 -6.66134e-16,

9.979684092131645)

In [11]:
# Step 3: Find rotation matrix to eliminate all but first k
# variables, where k = dim(nullspace(A)).
rotation = kernel_rotation(A)

18x18 Array{Float64,2}:
 -0.518625      0.195443     -0.0228639    …   3.88578e-16  -2.56739e-16
  0.518625     -0.195443      0.0228639       -4.996e-16     2.60209e-16
 -5.55112e-17   2.77556e-17   1.38778e-17     -9.48438e-17  -1.29654e-17
  0.259312     -0.0977215     0.0114319       -2.22616e-16   1.28555e-16
 -1.11022e-16   1.11022e-16  -1.38778e-17      1.98966e-17   1.07341e-17
 -0.199593     -0.497278      0.27661      …   0.284268      4.67942e-17
  0.199593      0.497278     -0.27661         -0.284268     -3.99154e-17
  6.10623e-16   3.1225e-17   -1.249e-16       -5.55112e-17  -4.58992e-17
  0.0997966     0.248639     -0.138305        -0.142134     -5.71159e-17
  0.0           0.0          -5.55112e-17      0.0          -1.17511e-16
 -0.0807202    -0.279872     -0.561388     …   5.55112e-17  -0.210819   
  0.0807202     0.279872      0.561388        -1.66533e-16   0.210819   
  1.11022e-16  -5.55112e-16   1.38778e-16      3.88578e-16  -8.32667e-17
  0.0403601     0.139936   

In [ ]:
# Step 4: Rotate problem using matrix found in Step 3.
J = rotation*H
j = rotation*h
kj = kh